In [1]:
import numpy as np
from torchvision.datasets import CIFAR10
import torchvision
from torchvision import transforms
import torch, torch.nn as nn
import torch.nn.functional as F

In [2]:
# class ResidualBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
#         super(ResidualBlock, self).__init__()
#         self.conv1 = nn.Sequential(
#                         nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
#                         nn.BatchNorm2d(out_channels),
#                         nn.ReLU())
#         self.conv2 = nn.Sequential(
#                         nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
#                         nn.BatchNorm2d(out_channels))
#         self.downsample = downsample
#         self.relu = nn.ReLU()
#         self.out_channels = out_channels
        
#     def forward(self, x):
#         residual = x
#         out = self.conv1(x)
#         out = self.conv2(out)
#         if self.downsample:
#             residual = self.downsample(x)
#         out += residual
#         out = self.relu(out)
#         return out

In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                     padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [4]:
def _make_layer(block, inplanes,planes, blocks, stride=1):
    downsample = None  
    if stride != 1 or inplanes != planes:
        downsample = nn.Sequential(            
            nn.Conv2d(inplanes, planes, 1, stride, bias=False),
            nn.BatchNorm2d(planes),
        )
    layers = []
    layers.append(block(inplanes, planes, stride, downsample))
    inplanes = planes
    for _ in range(1, blocks):
        layers.append(block(inplanes, planes))
    return nn.Sequential(*layers)

In [5]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        super().__init__()
        
        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 , num_classes)


    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None  
   
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        
        self.inplanes = planes
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)           # 224x224
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)         # 112x112

        x = self.layer1(x)          # 56x56
        x = self.layer2(x)          # 28x28
        x = self.layer3(x)          # 14x14
        x = self.layer4(x)          # 7x7

        x = self.avgpool(x)         # 1x1
        x = torch.flatten(x, 1)     # remove 1 X 1 grid and make vector of tensor shape 
        x = self.fc(x)

        return x

In [6]:
def resnet34():
    layers=[3, 4, 6, 3]
    model = ResNet(BasicBlock, layers)
    return model

In [7]:
# model = ResNet(ResidualBlock, [5, 5, 5]).to(device)

In [8]:
# # A basic block as shown in Fig.3 (right) in the paper consists of two convolutional blocks, each followed by a Bach-Norm layer. 
# # Every basic block is shortcuted in ResNet architecture to construct f(x)+x module. 
# # Expansion for option 'A' in the paper is equal to identity with extra zero entries padded
# # for increasing dimensions between layers with different feature map size. This option introduces no extra parameter. 
# import torch.nn.init as init
 
# def _weights_init(m):
#     """
#         Initialization of CNN weights
#     """
#     classname = m.__class__.__name__
#     if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
#         init.kaiming_normal_(m.weight)

# class LambdaLayer(nn.Module):
#     """
#       Identity mapping between ResNet blocks with diffrenet size feature map
#     """
#     def __init__(self, lambd):
#         super(LambdaLayer, self).__init__()
#         self.lambd = lambd

#     def forward(self, x):
#         return self.lambd(x)

# class BasicBlock(nn.Module):
#     expansion = 1

#     def __init__(self, in_planes, planes, stride=1, option='A'):
#         super(BasicBlock, self).__init__()
#         self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(planes)
#         self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(planes)
#         self.shortcut = nn.Sequential()
#         if stride != 1 or in_planes != planes:
#             if option == 'A':
#                 """
#                 For CIFAR10 experiment, ResNet paper uses option A.
#                 """
#                 self.shortcut = LambdaLayer(lambda x:
#                                             F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
#             elif option == 'B':
#                 self.shortcut = nn.Sequential(
#                      nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
#                      nn.BatchNorm2d(self.expansion * planes)
#                 )

#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))
#         out += self.shortcut(x)
#         out = F.relu(out)
#         return out


# class ResNet(nn.Module):
#     def __init__(self, block, num_blocks, num_classes=10):
#         super(ResNet, self).__init__()
#         self.in_planes = 16
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
#         self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
#         self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
#         self.linear = nn.Linear(64, num_classes)
#         self.apply(_weights_init)

#     def _make_layer(self, block, planes, num_blocks, stride):
#         strides = [stride] + [1]*(num_blocks-1)
#         layers = []
#         for stride in strides:
#             layers.append(block(self.in_planes, planes, stride))
#             self.in_planes = planes * block.expansion

#         return nn.Sequential(*layers)

#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.layer1(out)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         out = F.avg_pool2d(out, out.size()[3])
#         out = out.view(out.size(0), -1)
#         out = self.linear(out)
#         return out




# def resnet32():
#     return ResNet(BasicBlock, [5, 5, 5])


In [19]:
means = np.array((0.4914, 0.4822, 0.4465))
stds = np.array((0.2023, 0.1994, 0.2010))
# stds = np.array((0.247, 0.243, 0.261))
# transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomCrop(32, padding=4),
#     transforms.ToTensor(),
#     transforms.Normalize([0, 0, 0], [1, 1, 1])

# transform_augment = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomRotation([-30, 30]),
#     transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(means, stds),
# ])


transform_augment = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
    transforms.Resize((32, 32)),
    transforms.RandomRotation([-30, 30]),
#     transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
#     transforms.ColorJitter(brightness=0.02, contrast=0.001, saturation=0.2), # Set the color params
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(means, stds),
])



transform_augment_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(means, stds)])

batch_size = 400


train_loader = CIFAR10("./cifar_data/", train=True, transform=transform_augment)
trainloader = torch.utils.data.DataLoader(train_loader, 
                                          batch_size=batch_size,
                                          shuffle=True, 
                                          num_workers=2)


test_loader = CIFAR10("./cifar_data/", train=False, transform=transform_augment_test)
testloader = torch.utils.data.DataLoader(test_loader, 
                                         batch_size=batch_size,
                                         shuffle=False, 
                                         num_workers=2)

In [20]:
import sys
 
# setting path
sys.path.append('../')
from models import train_model, train_model_with_sc
model = resnet34()

In [22]:
device = ("cuda" if torch.cuda.is_available() else 'cpu')
model = model.to(device)


# optimizer = torch.optim.Adam(model.parameters(), lr= 0.001) # 0.001 !
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001, weight_decay=5e-4) # 0.001 !
criterion = torch.nn.CrossEntropyLoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=0.01,
#                       momentum=0.9, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# tb = SummaryWriter()
model, optimizer = train_model_with_sc(model, criterion, 
                    optimizer, train_dataloader=trainloader, test_dataloader=testloader, 
                    device=device,
                    n_epochs=1000, batch_size=batch_size, scheduler=scheduler)

model, optimizer = train_model(model, criterion, 
                    optimizer, train_dataloader=trainloader, test_dataloader=testloader, 
                    device=device,
                    n_epochs=1000, batch_size=batch_size)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f59c7b5ea60>
Traceback (most recent call last):
  File "/home/work/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/home/work/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f59c7b5ea60>
Traceback (most recent call last):
  File "/home/work/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/home/work/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_ali

INFO The mean accuracy test: 0.811
INFO -------------------------------
INFO Epoch 58
INFO The mean accuracy train: 0.878
INFO The mean accuracy test: 0.815
INFO -------------------------------
INFO Epoch 59
INFO The mean accuracy train: 0.874
INFO The mean accuracy test: 0.806
INFO -------------------------------
INFO Epoch 60
INFO The mean accuracy train: 0.878
INFO The mean accuracy test: 0.810
INFO -------------------------------
INFO Epoch 61
INFO The mean accuracy train: 0.880
INFO The mean accuracy test: 0.808
INFO -------------------------------
INFO Epoch 62
INFO The mean accuracy train: 0.881
INFO The mean accuracy test: 0.811
INFO -------------------------------
INFO Epoch 63
INFO The mean accuracy train: 0.882
INFO The mean accuracy test: 0.804
INFO -------------------------------
INFO Epoch 64
INFO The mean accuracy train: 0.880
INFO The mean accuracy test: 0.812
INFO -------------------------------
INFO Epoch 65
INFO The mean accuracy train: 0.883
INFO The mean accuracy t

INFO The mean accuracy test: 0.807
INFO -------------------------------
INFO Epoch 125
INFO The mean accuracy train: 0.906
INFO The mean accuracy test: 0.814
INFO -------------------------------
INFO Epoch 126
INFO The mean accuracy train: 0.907
INFO The mean accuracy test: 0.811
INFO -------------------------------
INFO Epoch 127
INFO The mean accuracy train: 0.908
INFO The mean accuracy test: 0.808
INFO -------------------------------
INFO Epoch 128
INFO The mean accuracy train: 0.908
INFO The mean accuracy test: 0.815
INFO -------------------------------
INFO Epoch 129
INFO The mean accuracy train: 0.912
INFO The mean accuracy test: 0.816
INFO -------------------------------
INFO Epoch 130
INFO The mean accuracy train: 0.911
INFO The mean accuracy test: 0.813
INFO -------------------------------
INFO Epoch 131
INFO The mean accuracy train: 0.912
INFO The mean accuracy test: 0.817
INFO -------------------------------
INFO Epoch 132
INFO The mean accuracy train: 0.911
INFO The mean ac

INFO The mean accuracy train: 0.919
INFO The mean accuracy test: 0.813
INFO -------------------------------
INFO Epoch 192
INFO The mean accuracy train: 0.920
INFO The mean accuracy test: 0.814
INFO -------------------------------
INFO Epoch 193
INFO The mean accuracy train: 0.918
INFO The mean accuracy test: 0.814
INFO -------------------------------
INFO Epoch 194
INFO The mean accuracy train: 0.919
INFO The mean accuracy test: 0.812
INFO -------------------------------
INFO Epoch 195
INFO The mean accuracy train: 0.918
INFO The mean accuracy test: 0.818
INFO -------------------------------
INFO Epoch 196
INFO The mean accuracy train: 0.918
INFO The mean accuracy test: 0.817
INFO -------------------------------
INFO Epoch 197
INFO The mean accuracy train: 0.919
INFO The mean accuracy test: 0.813
INFO -------------------------------
INFO Epoch 198
INFO The mean accuracy train: 0.920
INFO The mean accuracy test: 0.814
INFO -------------------------------
INFO Epoch 199
INFO The mean ac

INFO Epoch 258
INFO The mean accuracy train: 0.925
INFO The mean accuracy test: 0.811
INFO -------------------------------
INFO Epoch 259
INFO The mean accuracy train: 0.924
INFO The mean accuracy test: 0.810
INFO -------------------------------
INFO Epoch 260
INFO The mean accuracy train: 0.926
INFO The mean accuracy test: 0.812
INFO -------------------------------
INFO Epoch 261
INFO The mean accuracy train: 0.926
INFO The mean accuracy test: 0.812
INFO -------------------------------
INFO Epoch 262
INFO The mean accuracy train: 0.925
INFO The mean accuracy test: 0.812
INFO -------------------------------
INFO Epoch 263
INFO The mean accuracy train: 0.926
INFO The mean accuracy test: 0.811
INFO -------------------------------
INFO Epoch 264
INFO The mean accuracy train: 0.922
INFO The mean accuracy test: 0.804
INFO -------------------------------
INFO Epoch 265
INFO The mean accuracy train: 0.924
INFO The mean accuracy test: 0.810
INFO -------------------------------
INFO Epoch 266
I

...

INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.775
INFO -------------------------------
INFO Epoch 728
INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.783
INFO -------------------------------
INFO Epoch 729
INFO The mean accuracy train: 0.954
INFO The mean accuracy test: 0.775
INFO -------------------------------
INFO Epoch 730
INFO The mean accuracy train: 0.951
INFO The mean accuracy test: 0.775
INFO -------------------------------
INFO Epoch 731
INFO The mean accuracy train: 0.948
INFO The mean accuracy test: 0.780
INFO -------------------------------
INFO Epoch 732
INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.777
INFO -------------------------------
INFO Epoch 733
INFO The mean accuracy train: 0.953
INFO The mean accuracy test: 0.773
INFO -------------------------------
INFO Epoch 734
INFO The mean accuracy train: 0.949
INFO The mean accuracy test: 0.775
INFO -------------------------------
INFO Epoch 735
INFO The mean ac

INFO Epoch 794
INFO The mean accuracy train: 0.954
INFO The mean accuracy test: 0.773
INFO -------------------------------
INFO Epoch 795
INFO The mean accuracy train: 0.949
INFO The mean accuracy test: 0.776
INFO -------------------------------
INFO Epoch 796
INFO The mean accuracy train: 0.955
INFO The mean accuracy test: 0.764
INFO -------------------------------
INFO Epoch 797
INFO The mean accuracy train: 0.949
INFO The mean accuracy test: 0.769
INFO -------------------------------
INFO Epoch 798
INFO The mean accuracy train: 0.949
INFO The mean accuracy test: 0.771
INFO -------------------------------
INFO Epoch 799
INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.773
INFO -------------------------------
INFO Epoch 800
INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.773
INFO -------------------------------
INFO Epoch 801
INFO The mean accuracy train: 0.953
INFO The mean accuracy test: 0.776
INFO -------------------------------
INFO Epoch 802
I

INFO -------------------------------
INFO Epoch 861
INFO The mean accuracy train: 0.952
INFO The mean accuracy test: 0.777
INFO -------------------------------
INFO Epoch 862
INFO The mean accuracy train: 0.953
INFO The mean accuracy test: 0.772
INFO -------------------------------
INFO Epoch 863
INFO The mean accuracy train: 0.953
INFO The mean accuracy test: 0.772
INFO -------------------------------
INFO Epoch 864
INFO The mean accuracy train: 0.952
INFO The mean accuracy test: 0.770
INFO -------------------------------
INFO Epoch 865
INFO The mean accuracy train: 0.952
INFO The mean accuracy test: 0.775
INFO -------------------------------
INFO Epoch 866
INFO The mean accuracy train: 0.951
INFO The mean accuracy test: 0.770
INFO -------------------------------
INFO Epoch 867
INFO The mean accuracy train: 0.952
INFO The mean accuracy test: 0.776
INFO -------------------------------
INFO Epoch 868
INFO The mean accuracy train: 0.955
INFO The mean accuracy test: 0.770
INFO -----------

INFO The mean accuracy test: 0.770
INFO -------------------------------
INFO Epoch 928
INFO The mean accuracy train: 0.951
INFO The mean accuracy test: 0.778
INFO -------------------------------
INFO Epoch 929
INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.771
INFO -------------------------------
INFO Epoch 930
INFO The mean accuracy train: 0.951
INFO The mean accuracy test: 0.781
INFO -------------------------------
INFO Epoch 931
INFO The mean accuracy train: 0.951
INFO The mean accuracy test: 0.772
INFO -------------------------------
INFO Epoch 932
INFO The mean accuracy train: 0.954
INFO The mean accuracy test: 0.771
INFO -------------------------------
INFO Epoch 933
INFO The mean accuracy train: 0.954
INFO The mean accuracy test: 0.773
INFO -------------------------------
INFO Epoch 934
INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.776
INFO -------------------------------
INFO Epoch 935
INFO The mean accuracy train: 0.949
INFO The mean ac

INFO The mean accuracy train: 0.952
INFO The mean accuracy test: 0.776
INFO -------------------------------
INFO Epoch 995
INFO The mean accuracy train: 0.953
INFO The mean accuracy test: 0.773
INFO -------------------------------
INFO Epoch 996
INFO The mean accuracy train: 0.955
INFO The mean accuracy test: 0.772
INFO -------------------------------
INFO Epoch 997
INFO The mean accuracy train: 0.950
INFO The mean accuracy test: 0.771
INFO -------------------------------
INFO Epoch 998
INFO The mean accuracy train: 0.953
INFO The mean accuracy test: 0.774
INFO -------------------------------
INFO Epoch 999
INFO The mean accuracy train: 0.954
INFO The mean accuracy test: 0.767
INFO -------------------------------


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of test images: {100 * correct // total} %')

In [23]:
checkpoint = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()}
torch.save(checkpoint, 'resnet.pth')